# JETANK 색상 추적

이 문서는 기본적으로 밝은 노란색 객체를 추적하는 색상 추적 기능을 구현하는 데 사용됩니다.

**참고: 이 문서에는 서보 컨트롤이 포함되어 있습니다. 코드를 실행할 때 로봇 팔의 움직임 범위에 취약한 물체가 있는지 주의하고 어린이로부터 멀리 떨어지도록 하십시오.*

# 카메라 기능 라이브러리 가져오기

다음 코드 블록을 실행한 후 잠시 기다렸다가 카메라가 초기화될 때까지 기다립니다. 초기화가 성공하면 코드 블록 아래에 300x300 크기의 실시간 비디오 화면이 나타납니다.

이 화면을 마우스 오른쪽 단추로 클릭한 후 출력용 새 보기 만들기를 클릭하면 다시 창에 카메라 화면을 넣을 수 있습니다. 문서의 다른 부분을 탐색하더라도 언제든지 카메라 화면을 볼 수 있습니다. 이 방법은 다른 위젯에도 적용됩니다.

이 코드 블록을 여러 번 실행하면 초기화에 실패할 수 있습니다. 이 솔루션은 이미 jetbot.Camera에 포함되어 있습니다. 커널을 다시 시작하기만 하면 되지만 탭 위의 원 화살표를 사용하지 않도록 주의하십시오. 그래도 초기화에 실패할 수 있습니다.

커널을 재시작하는 것은 권장되는 방법입니다:
왼쪽에 있는 파일 브라우저에서 녹색 점이 앞에 있는*.ipynb 파일을 마우스 오른쪽 버튼으로 클릭하고(녹색 점은 커널이 실행 중임을 나타냅니다), Shut Down Kernel을 선택하면 녹색 점이 사라진 다음 이 탭을 닫고 방금 닫았던*.ipynb 파일을 더블 클릭하여 커널을 다시 시작합니다.

다음 코드를 다시 실행하면 카메라가 정상적으로 초기화됩니다.

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### TTLservo 클래스 가져오기

먼저 servos를 제어하는 데 사용되는 라이브러리를 가져와야 합니다. 이 코드를 실행하기 전에 ttYTHS1(Jetson Nano와 servos 간의 통신을 위한 직렬 포트)은 기본적으로 Jetson Nano 시스템 자체에서 점유하고 있습니다. 설치 스크립트가 정상적으로 모든 것을 실행하면 이전에 ttYTHS1을 점유했던 서비스가 종료되고 스크립트는 자동으로 ttYTHS1에 0666 권한을 추가합니다.



이전 설치 스크립트를 실행하지 못하면 여기에서 tTYTHS1을 사용할 때 사용 권한 거부 오류가 발생할 수 있습니다. 이 오류가 발생하면 다음과 같은 해결 방법이 있습니다:
- JupyterLab의 왼쪽 상단에 있는 플러스 기호를 클릭하면 새 Launcher 탭이 열리고 이 탭의 Terminal이 Jetson Nano 콘솔이 열립니다.
- 콘솔에 다음을 입력하고 Enter 키를 누릅니다.
    > sudo chmod 777 /dev/ttyTHS1
- Jetpack의 기본 비밀번호 'jetbot'을 입력하고 Enter 키를 눌러 수정 권한을 확인합니다.
- 왼쪽의 'JEANK_1_servos.ipynb'를 마우스 오른쪽 버튼으로 클릭하고 'Shut Down Kernal'을 클릭합니다.
- 현재 'JEANK_1_servos.ipynb' 탭을 닫고 왼쪽 'JEANK_1_servos.ipynb'를 다시 두 번 클릭하여 새 Kernal을 실행합니다.

그런 다음 다음 다음 코드를 선택하고 Ctrl+Enter를 눌러 servos를 제어하는 데 사용되는 라이브러리를 가져올 수 있습니다. 다음 프롬프트가 나타납니다:
포트를 여는 데 성공했습니다
보레이트 변경 성공

In [2]:
from SCSCtrl import TTLServo
import time

# If the TTL servo communicates too frequently, 
# there is a certain probability that rx and tx communication errors will be reported.
# This defines a delay for a period of time after each communication to avoid excessive communication frequency.
servoCtrlTime = 0.001

# Turn the No. 1 and No. 5 servos to the initial position.
# servo No. 1 controls the PAN axis movement of the camera pan/tilt, turning it horizontally.
# servo No. 5 controls the TILT axis movement of the camera pan/tilt, and the upward and downward pitching in the numerical direction.
TTLServo.servoAngleCtrl(1, 0, 1, 150)
time.sleep(servoCtrlTime)
TTLServo.servoAngleCtrl(5, 0, 1, 150)
time.sleep(servoCtrlTime)

# camera looks up.
def cameraUp(speedInput):
    TTLServo.servoAngleCtrl(5, -70, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera looks down.
def cameraDown(speedInput):
    TTLServo.servoAngleCtrl(5, 25, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera looks right.
def ptRight(speedInput):
    TTLServo.servoAngleCtrl(1, 80, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera looks left.
def ptLeft(speedInput):
    TTLServo.servoAngleCtrl(1, -80, 1, speedInput)
    time.sleep(servoCtrlTime)

# camera tilt axis motion stops.
def tiltStop():
    TTLServo.servoStop(5)
    time.sleep(servoCtrlTime)

# camera pan axis motion stops.
def panStop():
    TTLServo.servoStop(1)
    time.sleep(servoCtrlTime)

# After running the above code block, the No. 1 and No. 5 servo of the camera pan/tilt will slowly rotate to the middle position.

Succeeded to open the port
Succeeded to change the baudrate


# 색상 인식 및 추적 기능

ColorSelect에서는 개체 색상의 HSV 값을 구하는 방법을 소개하고 최대값과 최소값을 기록했습니다. colorUpper와 colorLower에 최대값과 최소값을 할당하고 np.array([H, S, V]) 형식에 주의하십시오.

자신의 오브젝트 색상으로 대체하지 않으면 기본 프로그램을 사용하여 색상 인식을 구현할 수도 있습니다. 기본 추적 색상은 밝은 노란색 오브젝트입니다.

In [3]:
import cv2
import numpy as np

# Define the color that needs to be recognized.

#Yellow #FFFF00
colorUpper = np.array([44, 255, 255])
colorLower = np.array([24, 100, 100])

# Red FF0000
# colorUpper = np.array([180, 255, 255])
# colorLower = np.array([160, 100, 100])

# Green #00FF00
# colorUpper = np.array([50, 255, 255])
# colorLower = np.array([70, 200, 100])

# Blue #0000FF
# colorUpper = np.array([110, 225, 255])
# colorLower = np.array([135, 180, 200])

# Cyan #00FFFF
# colorUpper = np.array([80, 255, 255])
# colorLower = np.array([105, 180, 180])

# Magenta #FF00FF
# colorUpper = np.array([140, 255, 255])
# colorLower = np.array([160, 150, 200])


# Define the position tolerance of the camera when turning to this object.
# The higher the value, the higher the accuracy of the camera when aiming, 
# but too high a value may also cause the camera to continuously swing.
error_tor = 25

# This is the P value of the simple PID regulator, 
# which is the proportional adjustment coefficient of the motion speed.
# If this value is too high, it will cause the camera PT motion overshoot, 
# and if it is too low, it will cause the color tracking response speed to be too slow.
PID_P = 3

# Color recognition and tracking function.
def findColor(imageInput):
    # Convert video frames to HSV color space.
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # Create a mask for pixels that match the target color.
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    
    # Erode, this process will remove the relatively 
    # small area in the mask just selected, which can be understood as denoising.
    mask = cv2.erode(mask, None, iterations=2)
    
    # dilate, the corrosion process just now will cause the large area to become 
    # smaller and the small area to disappear. This step is to restore the large area to its previous size.
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Obtain the conformed area contour.
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
    
    # If there is a matching area, start to control the movement of the steering gear to achieve color tracking.
    if len(cnts) > 0:
        # Draw text to show that the target has been found.
        imageInput = cv2.putText(imageInput,'Target Detected',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        
        # Find the contour of the largest area.
        c = max(cnts, key=cv2.contourArea)
        
        # Get the location of the center point of this area and the radius of this area.
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        # X, Y are the center points of the area.
        X = int(box_x)
        Y = int(box_y)
        
        # error_X, error_Y are the absolute value of the error 
        # between the center point of the area and the center point of the frame.
        error_Y = abs(150 - Y)
        error_X = abs(150 - X)
        
        # Draw the size and position of this area.
        cv2.rectangle(imageInput,(int(box_x-radius),int(box_y+radius)),(int(box_x+radius),int(box_y-radius)),(255,255,255),1)
        
        if Y < 150 - error_tor:
            # Camera looks up.
            imageInput = cv2.putText(imageInput,'Looking Up',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            cameraUp(error_Y*PID_P)
        elif Y > 150 + error_tor:
            # Camera looks down.
            imageInput = cv2.putText(imageInput,'Looking Down',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            cameraDown(error_Y*PID_P)
        else:
            # The error in the vertical direction is less than the tolerance, 
            # the camera stops moving in the pitch direction.
            imageInput = cv2.putText(imageInput,'Y Axis Locked',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            tiltStop()

        if X < 150 - error_tor:
            # Camera looks left.
            imageInput = cv2.putText(imageInput,'Looking Left',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            ptLeft(error_X*PID_P)
        elif X > 150 + error_tor:
            # Camera looks right.
            imageInput = cv2.putText(imageInput,'Looking Right',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            ptRight(error_X*PID_P)
        else:
            # The error in the horizontal direction is less than the tolerance, 
            # and the camera stops moving in the horizontal direction.
            imageInput = cv2.putText(imageInput,'X Axis Locked',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            panStop()

    # If no area matching the target color is found, the camera stops rotating.
    else:
        imageInput = cv2.putText(imageInput,'Target Detecting',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        tiltStop()
        panStop()
    
    return imageInput

다음 코드를 실행해도 프레임이 변경되지 않습니다. image_widget.value 값이 변경되고 camera.observe() 함수가 호출되어야 최종 효과를 관찰할 수 있습니다. 전자의 값은 처리된 프레임을 표시하는 데 사용되고, 후자의 함수는 새 프레임이 수집된 직후 이미지 처리 관련 메서드를 호출하는 데 사용됩니다. 

다음 코드 블록을 실행하여 색상 인식 및 추적 기능을 켜십시오.
**참고: 이 문서에는 서보 컨트롤이 포함되어 있습니다. 코드를 실행할 때 로봇 팔의 움직임 범위에 취약한 물체가 있는지 주의하고 어린이로부터 멀리 떨어지도록 하십시오.*

In [4]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(findColor(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

# 이 기능 처리를 끄고 카메라를 중지합니다
다음 코드 블록을 실행하여 이미지 처리 기능을 끕니다.

In [5]:
camera.unobserve(execute, names='value')

time.sleep(1)

tiltStop()
panStop()

다시 말씀드리지만, 나중에 노트북에 있는 카메라를 사용할 수 있도록 카메라 커넥터를 제대로 닫도록 하겠습니다.

In [6]:
camera.stop()